Vamos carregar o dataset de características de imóveis vendidos disponíveis no sklearn e armazenar as variáveis explicativas e target

In [33]:
from sklearn.datasets import fetch_openml
X, y = fetch_openml(name='house_prices', as_frame = True, return_X_y=True)

In [34]:
X

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,None,Reg,Lvl,AllPub,...,0.0,0.0,None,None,None,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,None,Reg,Lvl,AllPub,...,0.0,0.0,None,None,None,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,None,IR1,Lvl,AllPub,...,0.0,0.0,None,None,None,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,None,IR1,Lvl,AllPub,...,0.0,0.0,None,None,None,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,None,IR1,Lvl,AllPub,...,0.0,0.0,None,None,None,0.0,12.0,2008.0,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456.0,60.0,RL,62.0,7917.0,Pave,None,Reg,Lvl,AllPub,...,0.0,0.0,None,None,None,0.0,8.0,2007.0,WD,Normal
1456,1457.0,20.0,RL,85.0,13175.0,Pave,None,Reg,Lvl,AllPub,...,0.0,0.0,None,MnPrv,None,0.0,2.0,2010.0,WD,Normal
1457,1458.0,70.0,RL,66.0,9042.0,Pave,None,Reg,Lvl,AllPub,...,0.0,0.0,None,GdPrv,Shed,2500.0,5.0,2010.0,WD,Normal
1458,1459.0,20.0,RL,68.0,9717.0,Pave,None,Reg,Lvl,AllPub,...,0.0,0.0,None,None,None,0.0,4.0,2010.0,WD,Normal


In [35]:
y

0       208500.0
1       181500.0
2       223500.0
3       140000.0
4       250000.0
          ...   
1455    175000.0
1456    210000.0
1457    266500.0
1458    142125.0
1459    147500.0
Name: SalePrice, Length: 1460, dtype: float64

Agora vamos carregar algumas colunas para incluir transformações e montar um pipeline de transformações para então aplicar uma regressão linear nelas

In [36]:
Xtr = X.loc[:,['LotArea','PoolArea','OverallQual','FullBath', 'MSZoning']]

Então, vamos criar uma classe com os métodos fit, transform e de inicialização. o método de inicialização terá como padrão o metodo de centralização como padrão, mas aceitará também o método de padronização.

In [37]:
from sklearn.base import BaseEstimator,TransformerMixin

class ScaleTransformer(BaseEstimator,TransformerMixin):
    def __init__(self, method='center'): 
        if method not in ('center','standardize'):
            raise ValueError('metodo não é center ou standardize.')
        self.method = method
    
    def fit(self, X , *args, **kwargs):
        self.means_ = X.mean(axis=0)
        if self.method == 'standardize':
            self.stds_ = X.std(axis=0)
        return self

    def transform(self, X):
        if self.method == 'standardize':
            return_frame = (X - self.means_) / self.stds_
        elif self.method == 'center':
            return_frame = (X - self.means_)
        return return_frame

Agora vamos criar o pipeline com a classe ScaleTransformer que criamos acima e também interações polinomiais.

In [38]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

pipe = Pipeline(steps=[
    ('centralizador', ScaleTransformer(method='standardize')),
    ('interacoes', PolynomialFeatures(interaction_only=True, include_bias= False))
]
)

Por fim, aplicamos às colunas que desejamos (LotArea e OverallQual) o pipe que definimos na célula acima, também aplicamos um OneHotEncoding para a variável MSZoning e damos simplesmente um pass through (não aplicar nenhuma transformação) na variável PoolArea.

In [39]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

pipeline1 = ColumnTransformer([
    ('interacoes', pipe, ['LotArea','OverallQual']),
    ('dummy_zoning', OneHotEncoder(drop='first'), ['MSZoning']),
    ('pool','passthrough', ['PoolArea'])
], 
remainder = 'drop'
)

In [40]:
from sklearn import set_config
set_config(display = 'diagram')

Representação visual do pipeline criado até agora:

In [41]:
pipeline1

ColumnTransformer(transformers=[('interacoes',
                                 Pipeline(steps=[('centralizador',
                                                  ScaleTransformer(method='standardize')),
                                                 ('interacoes',
                                                  PolynomialFeatures(include_bias=False,
                                                                     interaction_only=True))]),
                                 ['LotArea', 'OverallQual']),
                                ('dummy_zoning', OneHotEncoder(drop='first'),
                                 ['MSZoning']),
                                ('pool', 'passthrough', ['PoolArea'])])

Por fim, inlcuimos a regressão linear no nosso pipeline depois das transformações feitas em cada variável.

In [42]:
from sklearn.linear_model import LinearRegression

modelo = Pipeline([('transformacoes', pipeline1), ('modelo_linear', LinearRegression())])

Fit e representação visual final do pipeline:

In [43]:
modelo.fit(Xtr,y)

Pipeline(steps=[('transformacoes',
                 ColumnTransformer(transformers=[('interacoes',
                                                  Pipeline(steps=[('centralizador',
                                                                   ScaleTransformer(method='standardize')),
                                                                  ('interacoes',
                                                                   PolynomialFeatures(include_bias=False,
                                                                                      interaction_only=True))]),
                                                  ['LotArea', 'OverallQual']),
                                                 ('dummy_zoning',
                                                  OneHotEncoder(drop='first'),
                                                  ['MSZoning']),
                                                 ('pool', 'passthrough',
                                                  ['PoolArea'])])),
                ('modelo_linear', LinearRegression())])

Coeficientes da regressão linear para cada varíavel:

In [44]:
modelo.steps[1][1].coef_

array([ 1.16327652e+04,  5.93790554e+04,  4.57863576e+03,  2.55330893e+03,
        9.49579863e+02,  1.55003347e+04, -1.20825605e+04,  3.82612215e+01])

Vamos aplicar a validação cruzada para o erro quadrático médio negativo, de forma a descobrir qual foi o erro das nossas previsões.

In [45]:
from sklearn.model_selection import cross_val_score
import numpy as np


np.sqrt(-cross_val_score(modelo,Xtr,y,cv=10,scoring='neg_mean_squared_error').mean()).round(0)

46187.0

Portanto, o erro médio do modelo em que aplicamos as transformações é de 46187 dólares.

In [46]:
pipeline2 = ColumnTransformer([
    ('interacoes', 'passthrough', ['LotArea','OverallQual']),
    ('dummy_zoning', OneHotEncoder(drop='first'), ['MSZoning']),
    ('pool','passthrough', ['PoolArea'])
], 
remainder = 'drop'
)

modelo2 = Pipeline([('transformacoes', pipeline2), ('modelo_linear', LinearRegression())])

In [47]:
np.sqrt(-cross_val_score(modelo2,Xtr,y,cv=10,scoring='neg_mean_squared_error').mean()).round(0)

46031.0

Já o erro médio do modelo em que não aplicamos nenhuma transformação no pipeline, exceto o OneHotEncoder, é de 46031 dólares e portanto menor. Nesse caso então as transformações definidas no primeiro pipeline, que incluem a transformação de escala e interações polinomiais, geraram apenas um erro maior nesse modelo específico.

In [50]:
cross_val_score(modelo, Xtr, y, cv=10, scoring = 'r2').mean()

0.6651431874099518

In [51]:
cross_val_score(modelo2, Xtr, y, cv=10, scoring = 'r2').mean()

0.6656837187462855

Porém, quando verificamos o r² vemos que os dois modelos são virtualmente iguais.